In [230]:
import pandas as pd
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)
import numpy as np
import spacy
import contextualSpellCheck
import nltk

In [149]:
import glob
import os
import re

In [47]:
# bas = pd.read_csv("lyrics_all/Bas_lyrics.txt", header=None, sep='\n', error_bad_lines=False)[0].str.cat(sep=" ")
# fileCounter = len(glob.glob1("lyrics_all/","*.txt"))
# d = {}
# # for x in range(1, fileCounter):
# #     d["string{0}".format(x)] = "Hello"

b'Skipping line 333: expected 1 fields, saw 2\nSkipping line 2555: expected 1 fields, saw 2\n'


### Ingestion

In [207]:
from pathlib import Path

path = Path('/Users/nk/Documents/Metis/05 Unsupervised/Project/lyrics_all')

d = {}

for file in path.glob('*.txt'):
    with open (file, 'r', encoding='utf-8', errors='ignore') as f:
        lyric = f.read()
    if "Wu-Tang" in os.path.basename(file):
        d[os.path.basename(file).replace('.txt','')] = lyric.replace('\n',',').replace(',,','\n\n')
    else:
        d[os.path.basename(file).replace('_lyrics.txt','')] = lyric.replace('\n',',').replace(',,','\n\n')
    
# type(os.path.basename(file))
# print(d.keys())
#     with open (file, 'r', encoding='latin1') as f:

In [208]:
drake_list = pd.read_csv("lyrics_all/Drake.csv").lyrics.tolist()

In [209]:
drake_lyrics = '\n\n'.join(drake_list)

In [210]:
d['Drake'] = drake_lyrics

In [211]:
d.keys()

dict_keys(['Talib Kweli', 'CunninLynguists', 'Kanye West', 'Deniro Farrar', 'Eminem', 'Ice Cube', 'Kendrick Lamar', 'Logic', 'Tupac1', 'J Cole', 'NF', 'Eazy-E', 'Joey Badass', 'Common', 'Scarface', 'Isaiah Rashad', 'Royce Da 59', 'Jay-z', 'Chance The Rapper', 'Immortal Technique', 'Lupe Fiasco', 'Big L', 'Montana of 300', 'ASAP Rocky', 'Bas', 'The Notorious BIG', 'Wu-Tang Clan', 'Earl Sweatshirt', 'Lil Wayne', 'ASAP Ant', 'Drake', 'Tyler The Creator', 'Nas', 'Mac Miller', 'Andre 3000', 'Pusha-T', 'Tupac2', 'Action Bronson', 'Childish Gambino'])

In [212]:
data = []

for key in d.keys():
    for i in d[key].split('\n\n'):
        data.append([key,i])

In [213]:
df = pd.DataFrame()
df = df.append(data, False)
df.rename(columns={0:"artist",1:"lyrics"}, inplace=True)

In [214]:
df.loc[13389][1]
df.head()

,artist,lyrics
0,Talib Kweli,",We sell crack to our own out the back of our homes,We smell the musk of the dusk in the crack of the dawn,We go through episodes too, like Attack of the Clones,Work 'til we break our back and you hear the crack of the bone,Just to get by, just to get by, just to get by, just to get by,We commute to computers,Spirits stay mute while you egos spread rumors,We're survivalists turned to consumers..."
1,Talib Kweli,"This morning, I woke up,Feelin' brand new, I jumped up,Feelin' my highs and my lows,In my soul and my goals,Just to stop smokin' and stop drinkin',I've been thinkin' I've got my reasons,Just to get by, just to get by,Just to get by, just to get by"
2,Talib Kweli,"We keepin' it gangster, say fo shizzle, fo sheezy, and stayin' crunk,It's easy to pull a breezy, smoke trees, and we stayin' drunk,Yo, I activism, attackin' the system, the Blacks and Latins in prison,Numbers have risen, they're victims lackin' the vision,Shit, and all they got is rappin' to listen to,I let them know we missin' you, the love is unconditional,Even when the condition is critical..."
3,Talib Kweli,"This morning, I woke up,Feelin' brand new, I jumped up,Feelin' my highs and my lows,In my soul and my goals,Just to stop smokin' and stop drinkin',And I've been thinkin' I've got my reasons,Just to get by, just to get by,Just to get by, just to get by"
4,Talib Kweli,"Yoyoyo, yo, yo,Some people cry, and some people try,Just to get by, for a piece of the pie,You love to eat and get high,We deceive when we lie, and we keepin' it fly,When the people decide to keep a disguise,Can't see they eyes, see the evil inside,But there's people you find,Strong or feeble in mind, I stay readin' the signs,Some people cry, and some people try,Just to get by, for a piece of ..."


In [298]:
df.to_csv("data_compiled_raw.csv")

### Preprocessing

In [215]:
df["cleaned"] = df.lyrics.replace({',':' '}, regex=True).str.strip().apply(lambda x: re.sub('^\\[(.*?)\\]', '', x))
df["cleaned"] = df.cleaned.replace({"in'":'ing'}, regex=True)

In [216]:
df.drop_duplicates(subset=["cleaned"],inplace=True)

### DROPPING CHORUS IF IT HAS SLIGHT DEVIATION - STOPWORD THEN DROP?

In [217]:
df.head()

,artist,lyrics,cleaned
0,Talib Kweli,",We sell crack to our own out the back of our homes,We smell the musk of the dusk in the crack of the dawn,We go through episodes too, like Attack of the Clones,Work 'til we break our back and you hear the crack of the bone,Just to get by, just to get by, just to get by, just to get by,We commute to computers,Spirits stay mute while you egos spread rumors,We're survivalists turned to consumers...",We sell crack to our own out the back of our homes We smell the musk of the dusk in the crack of the dawn We go through episodes too like Attack of the Clones Work 'til we break our back and you hear the crack of the bone Just to get by just to get by just to get by just to get by We commute to computers Spirits stay mute while you egos spread rumors We're survivalists turned to consumers ...
1,Talib Kweli,"This morning, I woke up,Feelin' brand new, I jumped up,Feelin' my highs and my lows,In my soul and my goals,Just to stop smokin' and stop drinkin',I've been thinkin' I've got my reasons,Just to get by, just to get by,Just to get by, just to get by",This morning I woke up Feeling brand new I jumped up Feeling my highs and my lows In my soul and my goals Just to stop smoking and stop drinking I've been thinking I've got my reasons Just to get by just to get by Just to get by just to get by
2,Talib Kweli,"We keepin' it gangster, say fo shizzle, fo sheezy, and stayin' crunk,It's easy to pull a breezy, smoke trees, and we stayin' drunk,Yo, I activism, attackin' the system, the Blacks and Latins in prison,Numbers have risen, they're victims lackin' the vision,Shit, and all they got is rappin' to listen to,I let them know we missin' you, the love is unconditional,Even when the condition is critical...",We keeping it gangster say fo shizzle fo sheezy and staying crunk It's easy to pull a breezy smoke trees and we staying drunk Yo I activism attacking the system the Blacks and Latins in prison Numbers have risen they're victims lacking the vision Shit and all they got is rapping to listen to I let them know we missing you the love is unconditional Even when the condition is critical...
3,Talib Kweli,"This morning, I woke up,Feelin' brand new, I jumped up,Feelin' my highs and my lows,In my soul and my goals,Just to stop smokin' and stop drinkin',And I've been thinkin' I've got my reasons,Just to get by, just to get by,Just to get by, just to get by",This morning I woke up Feeling brand new I jumped up Feeling my highs and my lows In my soul and my goals Just to stop smoking and stop drinking And I've been thinking I've got my reasons Just to get by just to get by Just to get by just to get by
4,Talib Kweli,"Yoyoyo, yo, yo,Some people cry, and some people try,Just to get by, for a piece of the pie,You love to eat and get high,We deceive when we lie, and we keepin' it fly,When the people decide to keep a disguise,Can't see they eyes, see the evil inside,But there's people you find,Strong or feeble in mind, I stay readin' the signs,Some people cry, and some people try,Just to get by, for a piece of ...",Yoyoyo yo yo Some people cry and some people try Just to get by for a piece of the pie You love to eat and get high We deceive when we lie and we keeping it fly When the people decide to keep a disguise Can't see they eyes see the evil inside But there's people you find Strong or feeble in mind I stay reading the signs Some people cry and some people try Just to get by for a piece of ...


In [218]:
df["english_check"] = df.cleaned.map(lambda x: x.isascii())

In [221]:
pd.set_option('max_colwidth', 1000)
df[df.english_check == False]

,artist,lyrics,cleaned,english_check
4,Talib Kweli,"Yoyoyo, yo, yo,Some people cry, and some people try,Just to get by, for a piece of the pie,You love to eat and get high,We deceive when we lie, and we keepin' it fly,When the people decide to keep a disguise,Can't see they eyes, see the evil inside,But there's people you find,Strong or feeble in mind, I stay readin' the signs,Some people cry, and some people try,Just to get by, for a piece of the pie,You love to eat and get high,We deceive when we lie, and we keepin' it fly,When the people decide to keep a disguise,Can't see they eyes, see the evil inside,But there's people you find,Strong or feeble in mind, I stay readin' the signs,I rise as day breaks,Clouds hide the sun’s rays,Quick sand and mistakes,Smash walls to get through,I fight the good fight,Even on the bad days,It’s darkest before light,I’m gonna push through",Yoyoyo yo yo Some people cry and some people try Just to get by for a piece of the pie You love to eat and get high We deceive when we lie and we keeping it fly When the people decide to keep a disguise Can't see they eyes see the evil inside But there's people you find Strong or feeble in mind I stay reading the signs Some people cry and some people try Just to get by for a piece of the pie You love to eat and get high We deceive when we lie and we keeping it fly When the people decide to keep a disguise Can't see they eyes see the evil inside But there's people you find Strong or feeble in mind I stay reading the signs I rise as day breaks Clouds hide the sun’s rays Quick sand and mistakes Smash walls to get through I fight the good fight Even on the bad days It’s darkest before light I’m gonna push through,False
5,Talib Kweli,"Turbo-charged bars,Slot cars on tracks, falling off, game scars in these rap wars,Ain’t nothing band-aids won’t cover,Touring with 7D coverage,Live bands, two buses, my own cash,No budget, no major label fundings, Jets baby,We went from crawling to walking to running,You can’t say nothing, you under gag order,Hopping out of Audi’s hurting y’all with kicks, I’m the Transporter,Think about it, never mind, you ain’t gotta,I’m already on some whole other, word to yo mufuckin' self,Nobody else gon’ do it for ya, nobody else,Every now and then someone’ll offer ya help, but they ain’t doing nothing,'Til they really see you tryna do something, I ain’t bluffing,Kush puffing on you jive turkeys,Stuffing stockings with these gifts,Audible applaudables homie take a wiff",Turbo-charged bars Slot cars on tracks falling off game scars in these rap wars Ain’t nothing band-aids won’t cover Touring with 7D coverage Live bands two buses my own cash No budget no major label fundings Jets baby We went from crawling to walking to running You can’t say nothing you under gag order Hopping out of Audi’s hurting y’all with kicks I’m the Transporter Think about it never mind you ain’t gotta I’m already on some whole other word to yo mufucking self Nobody else gon’ do it for ya nobody else Every now and then someone’ll offer ya help but they ain’t doing nothing 'Til they really see you tryna do something I ain’t bluffing Kush puffing on you jive turkeys Stuffing stockings with these gifts Audible applaudables homie take a wiff,False
6,Talib Kweli,"I rise as day breaks,Clouds hide the sun’s rays,Quick sand and mistakes,Smash walls to get through,I fight the good fight,Even on the bad days,It’s darkest before light,I’m gonna push through",I rise as day breaks Clouds hide the sun’s rays Quick sand and mistakes Smash walls to get through I fight the good fight Even on the bad days It’s darkest before light I’m gonna push through,False
7,Talib Kweli,"When you a man, action is what’s expected, nothing is spoken on,Producer never got time for no excuses, gotta go beyond,My flow is tom-foolery,I smoke these pork rappers so fast they call me charcuterie,I beautifully exude the vibe that’s free of ambiguity,Ya goonery for the sake of goonery is cartoons to me,It’s coone

In [297]:
df.to_csv('data_initial.csv')

In [194]:
from sklearn.feature_extraction.text import CountVectorizer

In [283]:
type(X)

scipy.sparse.csr.csr_matrix

In [232]:
cv = CountVectorizer(stop_words='english')
# cv2 = CountVectorizer(ngram_range=(1,2), binary=True, stop_words='english')
X = cv.fit_transform(df.cleaned)

df_X = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())
df_X.iloc[:5,33665:33711]

/opt/anaconda3/envs/nlpenv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,zzzh,zöld,ándale,ángeles,ça,élysées,éses,état,étoiles,órale,ölén,úgy,œuvre,ʿalay,мишка,сould,億万長者少年倶楽部,끝내주는데,많나봐,멋있다,모임,백만장자,소년들의,스타일,신발,자기,진짜,𝘊𝘢𝘳𝘵𝘪,𝘗𝘭𝘢𝘺𝘣𝘰𝘪,𝘛𝘩𝘳𝘰𝘶𝘨𝘩,𝘢𝘥,𝘣𝘰𝘶𝘵,𝘤𝘰𝘶𝘯𝘵𝘦𝘳,𝘥𝘢𝘮𝘯,𝘩𝘶𝘯𝘥𝘳𝘦𝘥𝘴,𝘪𝘯,𝘪𝘵𝘢𝘭𝘪𝘤𝘴,𝘭𝘪𝘣𝘴,𝘮𝘰𝘯𝘦𝘺,𝘯𝘦𝘦𝘥,𝘰𝘷𝘦𝘳𝘥𝘰𝘴𝘦,𝘵𝘩𝘦,𝘵𝘰,𝘶𝘩,𝘸𝘪𝘵𝘩,𝘺𝘶𝘩
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# min_df?

### Baseline LDA

In [234]:
from sklearn.decomposition import LatentDirichletAllocation

In [239]:
lda.components_.shape

(20, 33711)

In [286]:
lyricsarray = df.cleaned.values

In [293]:
tf_vectorizer = CountVectorizer(stop_words = 'english',
                                lowercase = True)
#                                 token_pattern = r'\b[a-zA-Z]{3,}\b'
#                                 max_df = 0.5, 
#                                 min_df = 5)
dtm_tf = tf_vectorizer.fit_transform(lyricsarray)

lda = LatentDirichletAllocation(n_components=20, random_state=0)
lda.fit(dtm_tf)

LatentDirichletAllocation(n_components=20, random_state=0)

In [295]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf= TfidfVectorizer(stop_words='english', lowercase=True)
dtm_tf = tfidf.fit_transform(lyricsarray)
lda = LatentDirichletAllocation(n_components=20, random_state=0)
lda.fit(dtm_tf)

LatentDirichletAllocation(n_components=20, random_state=0)

In [245]:
import pyLDAvis
import pyLDAvis.sklearn

In [296]:
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda, dtm_tf, tf_vectorizer)


/opt/anaconda3/envs/nlpenv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/anaconda3/envs/nlpenv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/opt/anaconda3/envs/nlpenv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/nlpenv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.437122  0.004374       1        1  66.051279
1     -0.036903  0.134501       2        1   3.133278
6     -0.027074 -0.004287       3        1   2.128741
10    -0.023309 -0.007628       4        1   2.053889
7     -0.025329 -0.006498       5        1   1.915605
17    -0.028563 -0.008882       6        1   1.908424
8     -0.021528 -0.012813       7        1   1.835570
2     -0.022302 -0.011852       8        1   1.832484
15    -0.025569 -0.006752       9        1   1.729894
11    -0.021702 -0.012698      10        1   1.723707
13    -0.023618 -0.007109      11        1   1.713526
9     -0.019488 -0.007944      12        1   1.672648
14    -0.023760 -0.004462      13        1   1.633696
12    -0.017381 -0.008052      14        1   1.597459
5     -0.023109 -0.004301      15        1   1.585169
16    -0.023074 -0.007292      16        1   1.558675
3     -0.021653 -0.006683      17        1   1.557134
18    -0.022055 -0.006247      18        1   1.526974
4     -0.012791 -0.008417      19        1   1.466792
19    -0.017912 -0.006957      20        1   1.375057, topic_info=               Term       Freq      Total Category  logprob  loglift
20687            oh  99.000000  99.000000  Default  30.0000  30.0000
186            2pac  34.000000  34.000000  Default  29.0000  29.0000
16725            la  25.000000  25.000000  Default  28.0000  28.0000
31988         verse  26.000000  26.000000  Default  27.0000  27.0000
31080         tupac  22.000000  22.000000  Default  26.0000  26.0000
...             ...        ...        ...      ...      ...      ...
11731         forth   0.439044   4.994376  Topic20  -7.5490   1.8552
3195        blacker   0.351010   2.221820  Topic20  -7.7728   2.4414
21795  penitentiary   0.355073   4.569413  Topic20  -7.7612   1.7319
7053         cradle   0.328172   4.973903  Topic20  -7.8400   1.5683
25445      ruthless   0.326099   5.111436  Topic20  -7.8464   1.5346

[1018 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
49         9  0.808274   123
99        19  0.576575  1970
158       11  0.784533  21st
186        1  0.764118  2pac
186        4  0.088167  2pac
...      ...       ...   ...
33536     15  0.489890   yuh
33543      3  0.245204   yup
33543     12  0.245204   yup
33647      6  0.649423   zoo
33660     10  0.788538  zulu

[1030 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 7, 11, 8, 18, 9, 3, 16, 12, 14, 10, 15, 13, 6, 17, 4, 19, 5, 20])